In [1]:
import torch
import torch.nn as nn
from torchinfo import summary  # Убедитесь, что вы установили torchinfo

Чтобы извлечь информацию о модели из файла model.ckpt, который является моделью PyTorch, вы можете использовать несколько методов. Важно понимать, что model.ckpt может содержать как состояние модели (веса), так и дополнительные метаданные, такие как параметры обучения и информация о текущем состоянии обучения.
Пример кода для извлечения информации о модели
Ниже приведен пример кода, который демонстрирует, как загрузить модель из файла model.ckpt, получить информацию о ее архитектуре и параметрах, а также вывести сводку о модели.

In [2]:
# Определите вашу модель (или импортируйте её из другого файла)
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 47)  # 47 классов для EMNIST

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [3]:
# Загрузка модели
def load_model(model_path):
    model = CNNModel()  # Инициализация модели
    model.load_state_dict(torch.load(model_path))  # Загрузка весов модели
    model.eval()  # Переключение в режим оценки
    return model

In [4]:
# Вывод информации о модели
def print_model_summary(model):
    print(summary(model, input_size=(1, 1, 28, 28)))  # Предполагается вход размером (batch_size=1, channels=1, height=28, width=28)

In [7]:
model_path = 'myapp/model_from_archive.ckpt'  # Укажите путь к вашему файлу .ckpt

In [8]:
model = load_model(model_path)  # Загрузка модели

/tmp/ipykernel_23293/2245479562.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))  # Загрузка весов модели


RuntimeError: Error(s) in loading state_dict for CNNModel:
	Missing key(s) in state_dict: "conv1.weight", "conv1.bias", "conv2.weight", "conv2.bias", "fc1.weight", "fc1.bias", "fc2.weight", "fc2.bias". 
	Unexpected key(s) in state_dict: "fc.0.weight", "fc.0.bias", "fc.3.weight", "fc.3.bias", "conv1.0.weight", "conv1.0.bias", "conv2.0.weight", "conv2.0.bias". 

In [ ]:
print_model_summary(model)  # Вывод информации о модели

Ошибка, которую вы видите, связана с предупреждением о безопасности при использовании функции `torch.load` в PyTorch. Это предупреждение указывает на то, что в будущем значение по умолчанию для параметра `weights_only` будет изменено на `True`, что повысит безопасность загрузки моделей, предотвращая выполнение произвольного кода.

### Анализ предупреждения

**Предупреждение**:
```
FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling...
```

Это предупреждение говорит о том, что использование `weights_only=False` может быть небезопасным, так как это позволяет загружать произвольные объекты. Рекомендуется использовать `weights_only=True`, если вы не контролируете источник загружаемых файлов.

### Как исправить предупреждение

1. **Используйте `weights_only=True`**:
   Если вы уверены, что файл содержит только веса модели и не содержит других объектов, вы можете изменить строку загрузки модели следующим образом:

   ```python
   self.model.load_state_dict(torch.load(model_path, weights_only=True))
   ```

   Однако если ваш файл содержит дополнительные данные (например, состояние оптимизатора или номер эпохи), вы не сможете использовать этот параметр.

2. **Обработка ошибок**:
   Если вы хотите продолжить обучение даже при возникновении этого предупреждения, вы можете игнорировать его с помощью модуля `warnings`:

   ```python
   import warnings
   warnings.filterwarnings("ignore", category=FutureWarning)
   ```

### Пример кода

Вот пример кода с учетом всех рекомендаций:

```python
import os
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)  # Игнорировать предупреждения FutureWarning

path = os.path.dirname(os.path.abspath(__file__))  # Текущий путь к файлу

class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 47)  # 47 классов для EMNIST

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

class Model:
    def __init__(self):
        model_path = os.path.join(path, 'model.ckpt')
        self.model = CNNModel()  # Инициализация модели
        try:
            # Загрузка весов модели с использованием weights_only=True
            self.model.load_state_dict(torch.load(model_path, weights_only=True))
            self.model.eval()  # Установка режима оценки
        except Exception as e:
            print(f"Error loading model: {e}")

    def predict(self, image):
        # Метод для предсказания класса изображения
        pass  # Реализуйте метод предсказания

if __name__ == "__main__":
    model = Model()  # Загрузка модели
```

### Заключение

Теперь ваш код будет игнорировать предупреждения о безопасности при загрузке модели. Однако важно помнить о рисках при использовании `torch.load` с параметром `weights_only=False`. Если у вас есть дополнительные вопросы или вам нужна помощь с конкретными аспектами работы с PyTorch или вашей моделью, пожалуйста, дайте знать!

Citations:
[1] https://fair-chem.github.io/tutorials/advanced/fine-tuning-in-python.html
[2] https://github.com/ultralytics/ultralytics/issues/14994
[3] https://community.openai.com/t/about-whisper-open-source-and-weights-only-false/993868
[4] https://discuss.pytorch.org/t/cant-set-weights-only-true-if-the-device-is-gpu/211604
[5] https://github.com/JaidedAI/EasyOCR/issues/1297
[6] https://discuss.pytorch.org/t/size-mismatch-for-model/212435
[7] https://stackoverflow.com/questions/78983479/how-can-i-continue-training-from-the-checkpoints-saved-in-the-previous-training